In [1]:
import requests
import re
import pandas as pd
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup

In [2]:
url_list = [f'https://www.khug.or.kr/jeonse/web/s07/s070102.jsp?cur_page={i}' for i in range(1, 71)]

# url에서 가져온 html에서 class_name이 'mb d_list'인 table을 찾아서 df로 만들고 리스트를 concat\
df_list = []
for url in tqdm(url_list):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table')
    for i, tr in enumerate(table.find_all('tr')):
        if i == 0:
            columns = [th.text for th in tr.find_all('th')]+['안심전세포털']
            df_tmp = pd.DataFrame(columns=columns)
        else:
            values = [td.text for td in tr.find_all('td')]
            link = 'https://www.khug.or.kr/jeonse/web/s07/'+tr.find('a')['href']
            values.append(link)
            df_tmp = pd.concat([df_tmp, pd.DataFrame([values], columns=columns)], ignore_index=True)
    df_list.append(df_tmp)

df = pd.concat(df_list, ignore_index=True)
df.drop(columns=['청약 접수기간', '번호', '공고일자'], inplace=True)
df.rename(columns={'전용면적(m2)':'전용면적','임대보증금액':'보증금'}, inplace=True)
df = df.applymap(lambda x: x.replace('  ', '').strip() if isinstance(x, str) else x)

  0%|          | 0/70 [00:00<?, ?it/s]

C:\Users\rmstn\AppData\Local\Temp\ipykernel_23592\1210179502.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('  ', '').strip() if isinstance(x, str) else x)


In [ ]:
df['신청자수'] = df['신청자수'].astype(int)
df[(df['시도']=='서울특별시')&(df['전용면적']>='40')].sort_values('신청자수', ascending=False).head(20)[df.columns[:8]]

,시도,시군구,주소,주택유형,매입유형,전용면적,보증금,신청자수
689,서울특별시,서울 양천구,서울 양천구 목동 713-9 에이앤캐슬 4층 404호,오피스텔(주거용),경매등매입형,52.49,"250,200,000",19
685,서울특별시,서울 강서구,서울 강서구 화곡동 346-57 마크하우스 6층 601호,다세대주택,경매등매입형,39.0,"247,500,000",5
671,서울특별시,서울 양천구,"서울 양천구 신정동 1000-12, 1000-13 건향파크빌8차 5층 501호",다세대주택,경매등매입형,33.7,"234,900,000",4
618,서울특별시,서울 강서구,"서울 강서구 화곡동 875-9, 875-10 아줄리움 6층 603호",다세대주택,경매등매입형,36.53,"207,000,000",0


In [10]:
df = df[['시도','시군구','주소','주택유형','매입유형','전용면적','보증금','안심전세포털']]
df.to_excel('rent_house_list.xlsx', index=False)